In [91]:
from google.cloud import bigquery
from google.oauth2 import service_account
import matplotlib.pyplot as plt
from astropy import units as u
from astropy.coordinates import SkyCoord
from astropy.coordinates import  FK4
import astropy.coordinates as coord
import pandas as pd
import numpy as np
import plotly.graph_objects as go
from ipywidgets import interact
from statsmodels.nonparametric.smoothers_lowess import lowess

In [84]:
# configs
dir = r'S:\Área de Trabalho\Artigo Massa DR3\BigQuery\gaia-316800-e305ca62954b.json'
project_name = 'gaia-316800'

In [85]:
# run client
credentials = service_account.Credentials.from_service_account_file(dir)
client = bigquery.Client(credentials=credentials, project=project_name)

In [86]:
# query fundamental parameters
query_parameters = '''
SELECT Cluster, RA_ICRS, DE_ICRS, dist, age, Av, alpha_high_int, 
alpha_low_int, Mc_int, mass_total, bin_frac, mass_seg, mass_seg_pval, r95
FROM gaia-316800.fundamental_parameters.OCs_intergrated_mf
'''

In [87]:
query_job = client.query(query_parameters)
clusters = query_job.to_dataframe()

In [88]:
# SkyCoord astropy

sc = SkyCoord(ra=(clusters['RA_ICRS'])*u.deg, dec=(clusters['DE_ICRS'])*u.deg, 
              distance=(clusters['dist'])*u.kpc, frame=FK4, obstime="J2000")


In [176]:
fig = go.Figure()

fig.add_trace(go.Scatter(
    x=sc.galactic.l.wrap_at(180*u.deg).radian, 
    y=sc.galactic.b.radian, 
    mode='markers',
    marker=dict(
        size=6,
        color=clusters['dist'],
        opacity=0.5,
        symbol='circle',
        colorscale='Viridis',
        showscale=True,
        colorbar=dict(title='log(age)')
    )
))

fig.update_layout(
    xaxis=dict(title='Galactic Longitude (Degrees)'),
    yaxis=dict(title='Galactic latitude (Degrees)'),
)

To contemplate the spatial distribution of clusters in our galaxy, we have this graph that was obtained in galactic coordinates using the numpy library.

To compose the agglomerates was used use of the high-quality membership of individual stars determined by the method described in detail in [Monteiro et al. (2020)](https://ui.adsabs.harvard.edu/abs/2020MNRAS.499.1874M/abstract) as well as determined fundamental parameters of open clusters as described in  [Dias et al. (2021)](https://ui.adsabs.harvard.edu/abs/2021MNRAS.504..356D/abstract) (and references therein) all applied to the high precision Gaia eDR3 data. With this information, we use a Monte-Carlo method that compares synthetic generated clusters to the observed ones to determine the properties of member stars based on a given set of theoretical isochrones. In this work, we use the Padova PARSEC version 1.2S database of stellar evolutionary tracks and isochrones [Bressan et al. 2012](https://ui.adsabs.harvard.edu/abs/2012MNRAS.427..127B/abstract), which is scaled to solar metal content with $Z_{\odot} = 0.0152$.

In [137]:
ind = (clusters['mass_seg_pval'] < 0.05)

X = np.log10(clusters['mass_total'][ind]).reset_index(drop=True)
Y = clusters['mass_seg'][ind].reset_index(drop=True)

# LOWESS curve
lowess_fit = lowess(Y, X)

# Calculate the mean and standard deviation
mean = lowess_fit[:, 1]
std = np.std(Y - lowess_fit[:, 1])

fig = go.Figure()

fig.add_trace(go.Scatter(x=lowess_fit[:, 0], 
                         y=lowess_fit[:, 1], mode='lines', line=dict(color='red'),
                         name='LOWESS'))

fig.add_trace(go.Scatter(x=lowess_fit[:, 0], 
                         y=mean - std, mode='lines', fill=None, 
                         line=dict(color='gray'), name='lower STD'))

fig.add_trace(go.Scatter(x=lowess_fit[:, 0],
                         y=mean + std, mode='lines', fill='tonexty', 
                         line=dict(color='gray'), name='upper STD'))

fig.add_trace(go.Scatter(x=X, y=Y, mode='markers', name= 'Clusters', marker=dict(
        size=8,
        color=clusters['dist'],
        opacity=1.0,
        symbol='circle',
        colorscale='Inferno',
        showscale=True,
        colorbar=dict(title='Dist.(kpc)', orientation='h'))))

fig.update_layout(
    xaxis=dict(title='log(age)'),
    yaxis=dict(title='KS test'),
)

Relationships that can generate insights into the galaxy were investigated. A sub sample based on the significance level of the Kolmogorov-Smirnov measure, taking only those clusters for which the distributions showed differences with a p-value lower than 0.05. The only parameter for which a significant correlation was observed was cluster total mass. The correlation for this parameter using the high significance sub sample is shown graphic, where a trend line based on a LOWESS regression is shown with its respective uncertainty according to the standard deviation.

In [175]:
fig = go.Figure()

fig.add_trace(go.Scatter(x = clusters['bin_frac'], 
                         y = clusters['dist'], mode='markers', name= 'Clusters', 
        marker=dict(
        size=(1/np.array([float(m) for m in clusters['mass_total']]))*260,
        sizemode='diameter',
        sizeref=0.1,
        color=clusters['dist'],
        opacity=0.8,
        symbol='circle',
        colorscale='Viridis',
        showscale=True,
        colorbar=dict(title='Dist.(kpc)', orientation='h'))))

fig.update_layout(
    xaxis=dict(title='Binary Fraction'),
    yaxis=dict(title='log(age)'),
)

With our database we can see a clear overestimation of the binary fraction with the increase of both distance and extinction as well as age. For clusters with log(age) < 9.0 and distance smaller than about 2.5 Kpc, the overestimation is about 20\%. This effect is a result mainly of the increased uncertainty in the fainter magnitudes and the smaller sampling of the main sequence region for clusters at higher ages and distances.